In [1]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from pprint import pprint
from selenium import webdriver          
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait   
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  

import warnings
warnings.filterwarnings('ignore')

### Задание №1

Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма)

Создадим тестовый аккаунт на mail.ru для выполнения домашнего задания nlitvak85@mail.ru с паролем к нему #5sQ5xtyDi#O

In [2]:
mail_server = "https://mail.ru/"
mail = "nlitvak85@mail.ru"
password = "#5sQ5xtyDi#O"

Заходим на сервер и получаем письма 

In [3]:
def receive_letters(mail_server, mail, password):
    letters_frame = pd.DataFrame(columns=['Author', 'Date', 'Link', 'Subj', 'Text'])
    driver = webdriver.Chrome() 
    driver.get(mail_server)

    assert "Mail.ru: почта" in driver.title
    driver.find_element_by_id("mailbox:login").send_keys(mail.split('@')[0])
    Select(driver.find_element_by_id('mailbox:domain')).select_by_visible_text('@'+mail.split('@')[1])
    driver.find_element_by_id("mailbox:password").send_keys(password)
    driver.find_element_by_id("mailbox:submit").click()

    wait = WebDriverWait(driver, 20);
    wait.until(EC.title_contains("Входящие"));
    assert "Входящие" in driver.title

    letters = driver.find_elements_by_xpath("//a[contains(@class,'llc js-tooltip-direction_letter-bottom js-letter-list-item llc_normal')]")

    i = 0
    for letter in letters:
        link = letter.get_attribute('href')
        info = letter.find_element_by_class_name("llc__content")
        author = info.find_element_by_class_name("ll-crpt").get_attribute('title')
        subj = info.find_element_by_class_name("ll-sj__normal").text
        date = letter.find_element_by_class_name("llc__item_date").get_attribute('title')
        letters_frame.loc[i] = [author, date, link, subj, "text"]
        i += 1
        
    i = 0    
    for i in range(len(letters_frame)):
        driver.get(letters_frame.loc[i, "Link"])
        wait.until(EC.presence_of_element_located((By.XPATH, "//*[contains(@id, '_BODY')]")))
        letters_frame.loc[i, "Text"] = driver.find_element_by_xpath("//*[contains(@id, '_BODY')]").text
        driver.back()
        wait.until(EC.title_contains("Входящие"));
        
    driver.quit()
    return letters_frame

In [4]:
letters_frame = receive_letters(mail_server, mail, password)

In [5]:
letters_frame

,Author,Date,Link,Subj,Text
0,Николай Литвак <iasha@mail.ru>,"Вчера, 23:17",https://e.mail.ru/inbox/1:22715d8a265a945c:0/?...,Тестовое письмо,Привет. \n\nЭто тестовое письмо для домашнего ...
1,Команда Почты Mail.ru <welcome@corp.mail.ru>,"Вчера, 23:15",https://e.mail.ru/inbox/1:6d96e36a14a405b1:0/?...,"Mail.ru – больше, чем почта. Познакомьтесь с п...","Здравствуйте, Николай!\n \n \n Mail.ru – ..."
2,Команда Почты Mail.ru <welcome@corp.mail.ru>,"Вчера, 23:15",https://e.mail.ru/inbox/1:b1313992a49332d8:0/?...,Как воспользоваться почтой с мобильного?,"Здравствуйте, Николай!\n \n \n Пользуйтесь ..."
3,Команда Mail.ru <welcome@corp.mail.ru>,"Вчера, 23:15",https://e.mail.ru/inbox/1:6f4207be3682cdae:0/?...,Узнайте о супервозможностях Почты Mail.ru,"\n Здравствуйте\n \n Спасибо, что\nзареги..."


После того как получили письма с mail.ru и вывели их на экран, теперь сохраним их в MongoDB

In [6]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client['letters']
letterdb = db.letters

In [7]:
for index, row in letters_frame.iterrows():
    letter_data={
        'Author': row['Author'],
        'Date': row['Date'],
        'Link': row['Link'],
        'Subj': row['Subj'],
        'Text': row['Text']
    }
    letterdb.insert(letter_data)

In [8]:
letterdb.letters

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'letters'), 'letters.letters')

Проверяем что данные записали в базу, вывод первые 5 записей

In [9]:
for letter in letterdb.find().limit(5):
    pprint(letter)

{'Author': 'Николай Литвак <iasha@mail.ru>',
 'Date': 'Вчера, 23:17',
 'Link': 'https://e.mail.ru/inbox/1:22715d8a265a945c:0/?back=1&afterReload=1',
 'Subj': 'Тестовое письмо',
 'Text': 'Привет. \n'
         '\n'
         'Это тестовое письмо для домашнего задания №6 в рамках урока Урок 6. '
         '"Selenium Для Python. Парсим динамические и закрытые данные" по '
         'предмету  "Методы сбора и обработки данных из сети Интернет"\n'
         '\n'
         '--\n'
         'Николай Литвак',
 '_id': ObjectId('5d2cf5c98c0fbf2105e3c451')}
{'Author': 'Команда Почты Mail.ru <welcome@corp.mail.ru>',
 'Date': 'Вчера, 23:15',
 'Link': 'https://e.mail.ru/inbox/1:6d96e36a14a405b1:0/?back=1&afterReload=1',
 'Subj': 'Mail.ru – больше, чем почта. Познакомьтесь с проектами Mail.ru Group',
 'Text': '    Здравствуйте, Николай!\n'
         ' \n'
         ' \n'
         '  Mail.ru – ваш проводник в интернет.\n'
         'Узнайте больше о проектах\n'
         'Mail.ru Group.\n'
         ' \n'
       

### Задание №2

Написать программу, которая собирает «Хиты продаж» с сайтов техники mvideo, onlinetrade и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары

Вместо абстрактного задания, решил сделать реальную бизнес задачу: необходимо сделать импорт товаров с сайта https://www.smeg.ru на сайт https://smg.moscow/

Каталог, который нас интересует находится по ссылке https://www.smeg.ru/katalog-produkcii/bytovaya-tehnika/

Ассортимент товаров в каждой категории достаточно большой и для удобства реализован постраничный вывод. Например, https://www.smeg.ru/duhovye-shkafy/, чтобы спарсить все товары сразу, необходимо нажать кнопку "все" и произойдет динамическая выгрузка всех товаров - что собственно нам и нужно. Дополнительно нужно разрешить использование куки сайта.

In [10]:
smeg_link = "https://www.smeg.ru/duhovye-shkafy/"

In [11]:
driver = webdriver.Chrome()
driver.get(smeg_link)

button_cookies = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'hide_cookie_message'))
) 
button_cookies.click()

button_all = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.LINK_TEXT, 'все'))
) 
button_all.click()

In [12]:
codes = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'code')))
items = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'item')))

goods_frame = pd.DataFrame(columns=['Code', 'Link', 'Image', 'Description'])

i = 0
for code in codes:
    goods_frame.loc[i] = [code.text, 'Link', 'Image', 'Description']
    i += 1
    
i = 0
for item in items:
    goods_frame.loc[i, "Link"] = item.find_element_by_tag_name('a').get_attribute('href')
    goods_frame.loc[i, "Image"] = item.find_element_by_tag_name('img').get_attribute('src')
    goods_frame.loc[i, "Description"] = item.find_element_by_tag_name('p').text
    i += 1   

driver.quit()

In [13]:
goods_frame

,Code,Link,Image,Description
0,SF800A,https://www.smeg.ru/product/sf800a/,https://www.smeg.ru/smeg_com/images/products/1...,"Многофункциональный духовой шкаф, 60 см, 6 фун..."
1,MP122S1,https://www.smeg.ru/product/mp122s1/,https://www.smeg.ru/smeg_com/images/products/1...,"Встраиваемая микроволновая печь, 60 см, высота..."
2,SF800PO,https://www.smeg.ru/product/sf800po/,https://www.smeg.ru/smeg_com/images/products/1...,"Многофункциональный духовой шкаф, 60 см, 6 фун..."
3,MP322X1,https://www.smeg.ru/product/mp322x1/,https://www.smeg.ru/smeg_com/images/products/1...,"Встраиваемая микроволновая печь, 60 см, высота..."
4,SFT805PO,https://www.smeg.ru/product/sft805po/,https://www.smeg.ru/smeg_com/images/products/1...,"Многофункциональный духовой шкаф, 60 см, 9 фун..."
5,SFP750POPZ,https://www.smeg.ru/product/sfp750popz/,https://www.smeg.ru/smeg_com/images/products/1...,Многофункциональный духовой шкаф с функцией пи...
6,SF750POL,https://www.smeg.ru/product/sf750pol/,https://www.smeg.ru/smeg_com/images/products/1...,"Многофункциональный духовой шкаф, 60 см, 11 фу..."
7,SF800GVPO,https://www.smeg.ru/product/sf800gvpo/,https://www.smeg.ru/smeg_com/images/products/1...,"Газовый духовой шкаф, 60 см, 8 функций, кремов..."
8,MP822PO,https://www.smeg.ru/product/mp822po/,https://www.smeg.ru/smeg_com/images/products/1...,"Встраиваемая микроволновая печь, 60 см, высота..."
9,SF750PO,https://www.smeg.ru/product/sf750po/,https://www.smeg.ru/smeg_com/images/products/1...,"Многофункциональный духовой шкаф, 60 см, 11 фу..."


После того как получили товары из категории и вывели их на экран, теперь сохраним их в MongoDB

In [14]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client['smeg']
smegdb = db.smeg

In [15]:
for index, row in goods_frame.iterrows():
    smeg_data={
        'Code': row['Code'],
        'Link': row['Link'],
        'Image': row['Image'],
        'Description': row['Description']
    }
    smegdb.insert(smeg_data)

In [16]:
smegdb.smeg

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'smeg'), 'smeg.smeg')

Проверяем что данные записали в базу, вывод первые 5 записей

In [17]:
for smeg in smegdb.find().limit(5):
    pprint(smeg)

{'Code': 'SF800A',
 'Description': 'Многофункциональный духовой шкаф, 60 см, 6 функций, антрацит, '
                'фурнитура позолоченная. Класс энергопотребления А.',
 'Image': 'https://www.smeg.ru/smeg_com/images/products/170/170/SF800A.jpg',
 'Link': 'https://www.smeg.ru/product/sf800a/',
 '_id': ObjectId('5d2d18858c0fbf2105e3c456')}
{'Code': 'MP122S1',
 'Description': 'Встраиваемая микроволновая печь, 60 см, высота 38 см, 6 '
                'функций, стекло Stopsol.',
 'Image': 'https://www.smeg.ru/smeg_com/images/products/170/170/MP122S1.jpg',
 'Link': 'https://www.smeg.ru/product/mp122s1/',
 '_id': ObjectId('5d2d18858c0fbf2105e3c457')}
{'Code': 'SF800PO',
 'Description': 'Многофункциональный духовой шкаф, 60 см, 6 функций, кремовый '
                ', фурнитура латунная.',
 'Image': 'https://www.smeg.ru/smeg_com/images/products/170/170/SF800PO.jpg',
 'Link': 'https://www.smeg.ru/product/sf800po/',
 '_id': ObjectId('5d2d18858c0fbf2105e3c458')}
{'Code': 'MP322X1',
 'Description

Это лишь пример по конкретной категории. Реально парсился весь каталог https://www.smeg.ru/ и перенесен на https://smg.moscow